In [192]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200



***Connection broken [Device not configured]
You may need to reconnect
Closing serial Serial<id=0x7f81d873f8e0, open=True>(port='/dev/tty.usbserial-0001', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [190]:
%sendtofile main.py --source main.py

Sent 63 lines (1230 bytes) to main.py.


In [91]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.envh")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [98]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]
api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)
runner.run()

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')


In [100]:
print(runner.output_buffer)
runner.stop()

[-0.04179526, -1.87689, -6.050216, -4.619785, -12.30089, -29.34657, -5.430662, 16.15306, 1.520026, 17.09361, 36.15711, 20.44824, -9.487555, -29.90106, -20.17232, -15.53988, -22.40484, 13.78118, 37.3049, 35.42534, 45.44753, 18.2136, 4.41837, -17.95101, -44.72243, -27.52366, -11.02454, 18.07533, 25.26217, 21.53173, 16.21344, 14.81441, -3.934866, -38.38978, -21.07615, 0.900435, 3.538435, -31.65422, -29.26047, -6.156304, 8.204639, 39.88416, 14.10834, 10.32538, 8.813786, -14.11746, -33.01486, -17.81416, 1.329798, -19.52213, 19.01212, 37.36596, 21.30258, 0.3455695, -27.02314, -29.83115, -23.26448, -1.170373, 21.85099, 28.86534, 6.050815, -15.00704, -33.18097, -20.95881, -0.03471351, -0.9796466, -0.3909593, 4.687751, -6.90556, -29.54284, -27.91251, -14.46558, 14.10185, 32.66275, 16.51373, 7.94457, 6.191947, -12.45231, -38.09886, -15.3043, -3.268488, -18.47307, 16.48532, 36.10726, 20.96221, -2.681967, -26.20954, -33.31726, -22.65532, -3.912046, 21.35466, 27.39192, 4.310492, -15.41527, -35.6973

In [78]:
f = open('lib/.envh', 'r')
print(f.read())

#.envh
WIFI_SSID=Rishil
WIFI_PASSWORD=rishilhotspot



In [134]:
%sendtofile main.py --source main.py

Sent 348 lines (6514 bytes) to main.py.


# Local functionality

In [51]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 30, in <module>
KeyboardInterrupt: 


# Local with downsample

In [125]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

# decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
# decode.setup() # setup peripherals and memory buffers
# decode.calibrate(calibration_data)

# print(4)

# decode.run()

# print(5)

# for i in range(5):
#     p13.on()
#     time.sleep(5)
#     print(decode.decoded_output)
#     p13.off()

# print(6)
# decode.stop()
# print(7)

....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
....62960
66368
.....62144
65552
....61312
64720
...3


In [126]:
decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
4
5


In [127]:
for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

.{12: 0.11574, 10: 0.1195, 7: 0.01818}
.{12: 0.02805, 10: 0.05897, 7: 0.01001}
.{12: 0.01652, 10: 0.08894, 7: 0.02123}
{12: 0.06124, 10: 0.04174, 7: 0.04992}
.{12: 0.0727, 10: 0.10921, 7: 0.02819}
6
7


# Online Functionality

In [201]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [198]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

# env_vars = load_env_vars("lib/.envh")
# # connect WiFI
# ssid = env_vars.get("WIFI_SSID")
# password = env_vars.get("WIFI_PASSWORD")
# ssid ="Rishil"
# password = "rishilhotspot"
# connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"
log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot_test_wireless')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


...

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 54, in <module>
KeyboardInterrupt: 


In [195]:
runner.stop()

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, i


[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missing-OK]  File "lib/decoding.py", line 145, in cca_eig

[missing-OK]ValueError: dimensions do not match

[missing-OK]Traceback (most recent call last):

[missing-OK]  File "lib/runner.py", line 144, in sample_callback

[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missing-OK]  File "lib/decoding.py", line 145, in cca_eig

[missing-OK]ValueError: dimensions do not match

[missing-OK]Traceback (most recent call last):

[missing-OK]  File "lib/runner.py", line 144, in sample_callback

[missing-OK]  File "lib/runner.py", line 109, in decode

[missing-OK]  File "lib/decoding.py", line 237, in classify

[missing-OK]  File "lib/decoding.py", line 41, in compute_corr

[missin

In [173]:
print(runner.decoded_output)

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, in sample_callback']
[leftinbuffer] ['  File "lib/runner.py", line 109, in decode']
[leftinbuffer] ['  File "lib/decoding.py", line 237, in classify']
[leftinbuffer] ['  File "lib/decoding.py", line 41, in compute_corr']
[leftinbuffer] ['  File "lib/decoding.py", line 145, in cca_eig']
[leftinbuffer] ['ValueError: dimensions do not match']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "lib/runner.py", line 144, i

# Online with downsample

In [180]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [179]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='downsample_test')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


....1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.....61136
65568
.....60064
64496
.....58976
63408
...3
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
connecting to network...
.network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [167]:
print(runner.decoded_output)
runner.stop()

[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 381, in OpenRequestJSONData']
[leftinbuffer] ['  File "lib/requests.py", line 282, in OpenRequest']
[leftinbuffer] ['Exception: Error to connect to 192.168.0.13:5001']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['64']
[leftinbuffer] ['1']
[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 381, in OpenRequestJSONData']
[leftinbuffer] ['  File "lib/requ

# Data send

In [208]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [153]:
%sendtofile main.py --source main.py

Sent 63 lines (1234 bytes) to main.py.


In [140]:
%sendtofile lib/runner.py --source lib/runner.py

Sent 251 lines (8444 bytes) to lib/runner.py.


In [205]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

# counter = 0
# while counter !=10:
#     if (counter % 2 == 0):
#         p26.on()
#         p13.off()
#     else:
#         p13.on()
#         p26.off()
#     time.sleep(1)
#     counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 3

api_host = "http://192.168.0.13:5001/lol"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='ssvep')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)

runner.run()

..ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [206]:
runner.stop()

[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/logging.py", line 131, in log']
[leftinbuffer] ['  File "lib/requests.py", line 163, in JSONRequest']
[leftinbuffer] ['  File "lib/requests.py", line 381, in OpenRequestJSONData']
[leftinbuffer] ['  File "lib/requests.py", line 282, in OpenRequest']
[leftinbuffer] ['Exception: Error to connect to 192.168.0.13:5001']


In [162]:
print(runner.output )

from .peripherals import PeripheralManager
from ulab import numpy as np
from logging import BaseLogger, MQTTLogger, HTTPLogger, logger_types

import gc
import config

from micropython import schedule


class Runner:
    def __init__(self, decoding_algo, buffer_size=256, stimulus_freqs=None) -> None:
        if stimulus_freqs is None:
            self.stim_freqs = config.STIM_FREQS  # assign defaults

        self.decoding_algo = decoding_algo

        self.base_sample_freq = config.ADC_SAMPLE_FREQ
        self.downsampled_freq = config.DOWNSAMPLED_FREQ

        self.preprocessing_enabled = config.PREPROCESSING

        if self.preprocessing_enabled:
            self.downsampled_freq = config.DOWNSAMPLED_FREQ

        self.sample_counter = 0
        self.buffer_size = buffer_size  # TODO: update this to be populated dynamically

        self.output_buffer = [0.0 for i in range(self.buffer_size)]
        self.decoded_output = {}

        self.is_setup = False
        self.is_sampling = F

# sending calibration data with posts

In [91]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [6]:
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [74]:
payload = {"b":[123]}#json.dumps({"a":[123]})
a = requests.JSONRequest("http://192.168.0.13:5001/", payload)
b = requests.JSONRequest("http://192.168.0.13:5001/calibration",payload)
print(a)
print(b)

.{'msg': 'data stored successfully'}
{'msg': 'successfully received'}


In [ ]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

time.sleep(5)

p26.off()
p13.off()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return data

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True
        
        gc.collect()
        trials.append(collectData(decode_period_s))
        
    runner.stop()

    gc.collect()
    p26.off()
    return trials

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

calibration_data[7] = getData(Nt, decode_period_s)
toSend = {"7": calibration_data[7]}
requests.JSONRequest("http://192.168.0.13:5001/7hz", toSend)
del calibration_data[7]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
toSend = {"10": calibration_data[10]}
requests.JSONRequest("http://192.168.0.13:5001/10hz", toSend)
del calibration_data[10]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
toSend = {"12": calibration_data[12]}
requests.JSONRequest("http://192.168.0.13:5001/12hz", toSend)
del calibration_data
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)



..1
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.connecting to network...
.network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
....29088
55264
.....37824
55264
.

In [51]:
gc.collect()

In [52]:
print(gc.mem_free())

55840


In [53]:
print(requests.JSONRequest("http://192.168.0.13:5001/calibration",json.dumps(calibration_data)))

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
MemoryError: memory allocation failed, allocating 9904 bytes


In [44]:
print(runner.is_sampling)

False


In [50]:
print(calibration_data)

{12: [[0.02507745, 2.52553, 12.85992, 11.96267, 0.6863966, 13.57261, -12.81222, -32.35726, 15.95395, 13.00588, -0.9205959, -22.48486, -2.830642, 39.04061, -16.44812, -25.8597, 0.6819317, 25.08447, -10.40546, -35.8309, 13.74132, 6.882522, 17.80202, -11.96199, -11.90709, 41.51297, 4.37162, -25.8305, -3.788256, 37.65075, 5.540895, -33.51657, -6.747506, 2.347939, 19.45399, -3.675349, -35.1315, 26.51662, 31.09047, -12.0729, -6.802375, -10.81737, 24.34443, 8.15633, -42.39853, 0.994298, 25.88819, 10.22811, -21.48804, -2.282226, 39.73905, 5.401531, -12.71161, -21.27685, -10.2993, 20.25273, -21.3752, -27.83162, 25.03197, 19.92385, -16.92972, -15.82081, 16.81641, 22.79442, -0.04126062, -2.785058, -14.77499, -24.10244, -18.88921, -3.194786, 32.29165, 17.0885, -29.07069, -2.497297, 12.36031, 11.33428, -10.74703, -29.77115, 20.25805, 31.61642, 0.7922319, -21.43603, 26.22578, 35.87283, -20.69857, -10.25543, 0.009931564, 21.30839, -3.036798, -38.82599, 6.201851, 29.423, 6.195831, -36.22295, 9.491094,